# Lesson 7 notes

### Check lesson7-resnet-mnist.ipynb
[Jump to lesson 7 on resnet-mnist](https://youtu.be/9spwoDYwW_I?t=127)

### Datablock - ImageItemList - from_folder()
```
il = ImageList.from_folder(path, convert_mode='L')
```

convert_mode='L' is passed to the library pillow, and open the image as black and white

## defaults.cmap

set the default color map for fastai

## split_by_folder
```
sd = il.split_by_folder(train='training', valid='testing')
```
you can use .no_split() if you don't want validation set.

cannot skip this method.

you can give the folder name on ='name' attribute.

## Transform on handwriting MNIST

we cannot flip it, cannot rotate, it will change the meaning.

we cannot zoom, it will be too fuzzy on this image size.

so just do padding.
```
tfms = ([*rand_pad(padding=3, size=28, mode='zeros')], [])
```
Also, no transform on validation set ( see [] )

## Number of channel in CNN
```
conv(1, 8) # 14
```
why 8 channel? you pick.

also, 14 is the size of the image on the layer, since it is stride=2

## Flatten()

The last of conv(16, 10) gives out a 3D tensor of 10 by 1 by 1.
But the loss function only take array ( array of 10 elements of popubilities for each digits ).

Flatten() remove those extra dimentions

### Pytorch module as function
```
model(xb)
```
We can use any pytorch.nn.module and called it as function, it will call forward() method for you. It is by design.

## Stride-2 vs Stride-1

Stride-1 does not change output activations size, stride-2 reduces by half. So you can add as many stride-1 layer as you like.

But it is not as simple. There are below problems

## Kaiming He paper
![](lesson7/lesson7-1.png)
Let's look at the training error of a network of 20 layers, and one with 56 layers.

56 layers one have a lot more stride-1 convolutions in the middle. So the one with more parameters should seriously overfit right? So you would expect the 56 layer one to zip down to zero-ish training error pretty quickly.

But this is not happen, it is actually worse.

## Kaiming He assumtion

He did not know why this problem occur, but he said if he can change the 56 layer a little bit, he can make it as good as the 20 layers one.

![](lesson7/lesson7-2.png)

For every 2 layer, instead of the normal output = c2(c1(x)), He change it to output = x + c2(c1(x)).

In other words, if the weights is zero, the layers becomes nothing happen at all, skip connection ( no change, pass-on ). That's why it is called identity x.


This is called a res-block. and the result is revolutionary.

## Why better result?
![](lesson7/lesson7-3.png)

The x and y is the weights' space, while z is the loss. without res-block, it is very bumpy.

## DenseNet
![](lesson7/lesson7-4.png)
So basically, instaed of x + c2(c1(x)), we concatenate it.

Since the dense-block is keeping the original pixel, it works well on segmentation. Because you want to reconstruct the original pixel of the picture

But going deeper every layer, since we concat it, the ouput get bigger in size. so it is very memory intensive.

TODO

## Segmentation

check lesson3-camvid-tiramisu.ipynb

### Fastai SegmentationItemList

It will automatically shows the colorcoded pixel on top of the image, where different color represent a class of object that its classified.

### How Segmentation works

In order for the model to classify a particular pixel to be a pedestrian, while another pixel is a bicyclist, It needs to know what a pedestrian looks like, in a broader sense. It needs to understand that this area of pixel belongs to a pedestrian.

It needs to really understand a lot about this picture before classifying each pixel.

We can get it work well by pre-training.

## U-Net
[Jump to lesson7 U-net lecture](https://youtu.be/9spwoDYwW_I?t=1860)
![](lesson7/lesson7-5.png)

It basically have 3 part. First part ( left side of U ) is to use maybe stride-2 conv to half-size the activations, while creating more channel ( remember, each face plane is a feature ).

Seoncd part ( bottom of U ) is to stop that stride-2 conv when the activation image hit a certain size ( let say 11 by 11 ). Don't go down to 1 by 1, this is not a classifier.

Thrid part ( right side of U ) is to re-construct a image by increasing the activation image size. We can do it by doing stride-half convolution, also known as a deconvolution, or transposed convolution.

## Stride-half Convolution - slow method
![](lesson7/lesson7-6.png)
The blue are inputs ( 2 by 2 ). It is padded to a imaginary 7 by 7 plane, as shown.

The shadow is the 3 by 3 kernal.

Performing image kernal computation, we got a 5 by 5 output ( the green part ).

But, it is very slow, a lot of computation. 

Also, the red circle part, it has a lot of padded zero, 8 or 7 out of 9 in the shadow is zero. It waste a lot of information.

## Stride-half Convolution - better method
![](lesson7/lesson7-7.png)
Instead of padding, just up-sample it.

It is called nearest neighbor interpolation.

Another simular method called bilinear interpolation. It is kind of like a weighted average for each cell.

## Copy and crop
![](lesson7/lesson7-8.png)
Even though we can use stride-half convolution, it is still impossible to get a image activation of 28 by 28, get generate back to 572 by 572, with that fine-details ( the color-coded the shape of pedestrian so well ).

There are also path ( copy and crop ), from each down-sampling on left, connected to right up-sampling. Think of it kind of like skip-connection in res-block.

Those copy and crop is concatenated into the up-sampling layer ( show as blue-framed white box )

### Check lecture on UNet fastai source code explanation
[Jump to Lecture on Unet source code](https://youtu.be/9spwoDYwW_I?t=2682)

## Image Restoration
Check lesson7-superres-gan.ipynb

Input with poor quality, low resolution, with words on the picutre, output with high quality, remove writing on it pictures.

## Crappify
Create poor quality photo, added random test on it.

## Fastai parallel()
It run a function in parallel.


### Importance on using pretrained model
```
.databunch(bs=bs).normalize(imagenet_stats, do_y=True))
```

the imagenet_stats identify that we are using the same stats as imagenet. Because we want to use their pretrained model.

The poor quality photo, that has text on it, in order to remove the test, we kind of need to know what a dog should look like under that text. That's why we use pretrain model.

## Unet_learner
```
unet_learner(data_gen, arch, wd=wd, blur=True, norm_type=NormType.Weight,
                         self_attention=True, y_range=y_range, loss_func=loss_gen)
```

blur, norm_type, and self_attention is very important for this generator type of things. Reasons will explain in part 2.


## MSELossFlat()

normal MSELoss takes array, this take a matrix, so it is good for image input.

### Affects on using MSELoss

The result of the generated image did a good job on removing the text on the picture, but did not do a good job on making better quality, It also failed to re-create the details on the eyes.

It is because the loss function we are using does not really describe what we want. The pixel differences of the original and the calculated is actually pretty small. But we are missing the texture of the cat's fur, and the eyeball too.



## Generative adversarial network
![](lesson7/lesson7-9.png)
A GAN tried to solve this problem by using a loss function that calls another model for you.

A Critic is another model, a binary classification model. It asks the input, is it a high-res cat? or a generated cat?

The Loss is now: how good are we fooling the critic?

It is a back and forth process. We first train some batches on the generator, using the critic as a loss function. The generator is going to become good at fooling the critic.

We then start training the critic on the newly generated images and stop the generator, the critic is going to be better.

We then train generator again with newly improved critic as a loss function again.


## Problem with GAN

If the Generator and the Critic is not using a pretrain model, it is very hard to train from start. It is like a blind leading a blind.

### Check lesson7-superres-gan.ipynb #Save generated images

## Free GPU memory
The following code can be run in notebook and you don't need to restart jupyter kernel.
```
learn = None
gc.collect()
```

## Notes on critic learner
```
loss_critic = AdaptiveLoss(nn.BCEWithLogitsLoss())
def create_critic_learner(data, metrics):
    return Learner(data, gan_critic(), metrics=metrics, loss_func=loss_critic, wd=wd)
```
We basically cannot use Resnet (maybe we can with spectral norm set in it ).

bascially: When doing a GAN, you need to be particularly careful that the generator and the critic can't kind of push into the same direction and like increase the weights out of control.

We have to use something called spectral normalization.

Reason will explain in part 2.

gan_critic() will give you a binary classifier suitable for GAN.


A GAN critic uses a slightly different way of averaging the different part of image when it does the loss, so anytime you are doing a GAN at the moment you have the wrap the function with AdaptiveLoss()


accuracy_thresh_expand is kind of the accuracy metrics but for GAN.

### Check lesson7-superres-gan.ipynb #GAN

```
learn = GANLearner.from_learners(learn_gen, learn_crit, weights_gen=(1.,50.), show_img=False, switcher=switcher,opt_func=partial(optim.Adam, betas=(0.,0.99)), wd=wd)
```

## Multiple loss function for GAN

```
weights_gen=(1.,50.)
```
We don't only use the critic as a loss function, otherwise the GAN could get very good at creating pictures that look like real pictures, but the picture actually have nothing to do with the original photo at all.

So we also added pixel loss as well.

The weights_gen I think it is passing which loss function we use more.

## Momentum on GAN
```
betas=(0.,0.99))
```
GAN hates momentum. And it doesn't make sense. As we keep switching between generator and critic.

0 is zero momentum

## Result of GAN learner
```
epoch	train_loss	gen_loss	disc_loss
1	2.071352	2.025429	4.047686
2	1.996251	1.850199	3.652173
3	2.001999	2.035176	3.612669
4	1.921844	1.931835	3.600355
5	1.987216	1.961323	3.606629
6	2.022372	2.102732	3.609494
7	1.900056	2.059208	3.581742
8	1.942305	1.965547	3.538015
9	1.954079	2.006257	3.593008
.
.
.
```

The result of the losses should is meaningless. Because we are switch between generator and critic, both should get better and better. So there should always be some losses ( loss should not keep going down )